<a href="https://colab.research.google.com/github/hanzlayyy/FYDP/blob/main/FYDP_Notebook_V1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv('/content/sample_data/data v1.csv')

In [3]:
df

,layer_height,infill_density,infill_pattern,nozzle_temperature,print_speed,material,roughness
0,0.02,90.0,grid,220.0,40.0,abs,25
1,0.02,90.0,honeycomb,225.0,40.0,abs,32
2,0.02,80.0,grid,230.0,40.0,abs,40
3,0.02,70.0,honeycomb,240.0,40.0,abs,68
4,0.02,90.0,grid,250.0,40.0,abs,92
...,...,...,...,...,...,...,...
544,0.30,10.0,rectilinear,215.0,120.0,abs,1.64
545,0.30,10.0,rectilinear,215.0,120.0,abs,1.4
546,0.30,10.0,rectilinear,215.0,90.0,abs,0.82
547,0.30,10.0,rectilinear,215.0,90.0,abs,0.76


In [4]:
df=df.drop(403, axis=0)

In [5]:
df['infill_pattern'].value_counts()

infill_pattern
grid           96
honeycomb      51
solid          50
zigzag         40
concentric     15
rectilinear    15
triangle       14
tetrahedral    12
line            9
line            3
Name: count, dtype: int64

In [6]:
df.isnull().mean()

layer_height          0.000000
infill_density        0.166058
infill_pattern        0.443431
nozzle_temperature    0.000000
print_speed           0.049270
material              0.049270
roughness             0.107664
dtype: float64

In [7]:
df = df.replace('line ', 'line')

In [8]:
missing = df[df.isnull().any(axis=1)]

In [9]:
zero_missing = df[~df.isnull().any(axis=1)]
#zero_missing = pd.get_dummies(zero_missing, columns=['infill_pattern','material'], dtype=int)


In [10]:
one_missing_infill_density = missing[missing['infill_density'].isnull() & ~missing[['infill_pattern','print_speed','material','roughness']].isnull().any(axis=1)]
#one_missing_infill_density = pd.get_dummies(one_missing_infill_density, columns=['infill_pattern','material'], dtype=int)

In [11]:
one_missing_infill_pattern = missing[missing['infill_pattern'].isnull() & ~missing[['infill_density','print_speed','material','roughness']].isnull().any(axis=1)]
#one_missing_infill_pattern = pd.get_dummies(one_missing_infill_pattern, columns=['material'], dtype=int)


In [12]:
one_missing_print_speed = missing[missing['print_speed'].isnull() & ~missing[['infill_density','infill_pattern','material','roughness']].isnull().any(axis=1)]
#one_missing_print_speed = pd.get_dummies(one_missing_print_speed, columns=['infill_pattern','material'], dtype=int)

In [13]:
one_missing_material = missing[missing['material'].isnull() & ~missing[['infill_density','infill_pattern','print_speed','roughness']].isnull().any(axis=1)]
#one_missing_material = pd.get_dummies(one_missing_material, columns=['infill_pattern'], dtype=int)

In [14]:
one_missing_roughness = missing[missing['roughness'].isnull() & ~missing[['infill_density','infill_pattern','print_speed','material']].isnull().any(axis=1)]
#one_missing_roughness = pd.get_dummies(one_missing_roughness, columns=['infill_pattern','material'], dtype=int)

In [15]:
combined = pd.concat([zero_missing, one_missing_infill_density], axis=0)

In [16]:
temp = pd.get_dummies(combined, columns=['infill_pattern','material'], dtype=int)

In [17]:
knn = KNNImputer(n_neighbors=3)
temp2 = pd.DataFrame(knn.fit_transform(temp))
temp2.columns = temp.columns
temp2

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_concentric,infill_pattern_grid,infill_pattern_honeycomb,infill_pattern_line,infill_pattern_rectilinear,infill_pattern_solid,infill_pattern_tetrahedral,infill_pattern_triangle,infill_pattern_zigzag,material_abs,material_pla,material_pla-aluminum
0,0.02,90.0,220.0,40.0,25.00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.02,90.0,225.0,40.0,32.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.02,80.0,230.0,40.0,40.00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.02,70.0,240.0,40.0,68.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.02,90.0,250.0,40.0,92.00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,0.15,10.0,225.0,80.0,16.17,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
274,0.15,10.0,225.0,80.0,17.12,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
275,0.15,10.0,225.0,80.0,12.75,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
276,0.15,10.0,225.0,80.0,18.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [18]:
zero_missing_1H  = temp2.copy()

In [19]:
zero_miss_WConc = zero_missing_1H.drop(columns = [ 'infill_pattern_grid','infill_pattern_honeycomb', 'infill_pattern_line',
       'infill_pattern_rectilinear', 'infill_pattern_solid','infill_pattern_tetrahedral', 'infill_pattern_triangle',
       'infill_pattern_zigzag'])
zero_miss_WConc

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_concentric,material_abs,material_pla,material_pla-aluminum
0,0.02,90.0,220.0,40.0,25.00,0.0,1.0,0.0,0.0
1,0.02,90.0,225.0,40.0,32.00,0.0,1.0,0.0,0.0
2,0.02,80.0,230.0,40.0,40.00,0.0,1.0,0.0,0.0
3,0.02,70.0,240.0,40.0,68.00,0.0,1.0,0.0,0.0
4,0.02,90.0,250.0,40.0,92.00,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
273,0.15,10.0,225.0,80.0,16.17,0.0,0.0,1.0,0.0
274,0.15,10.0,225.0,80.0,17.12,0.0,0.0,1.0,0.0
275,0.15,10.0,225.0,80.0,12.75,1.0,0.0,1.0,0.0
276,0.15,10.0,225.0,80.0,18.80,0.0,0.0,1.0,0.0


In [20]:
C =pd.get_dummies(one_missing_infill_pattern.drop(columns=['infill_pattern']), columns = ['material'],dtype = int)
C['infill_pattern_concentric']= np.NaN
C['material_pla-aluminum'] = 0

D=C.reindex(columns=list(zero_miss_WConc.columns))
D

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_concentric,material_abs,material_pla,material_pla-aluminum
190,0.20,30.0,210.0,30.0,42.7,NaN,0,1,0
191,0.15,20.0,190.0,75.0,26.2,NaN,0,1,0
192,0.20,10.0,190.0,30.0,54,NaN,0,1,0
193,0.20,20.0,200.0,75.0,30.5,NaN,0,1,0
194,0.10,10.0,190.0,120.0,25,NaN,0,1,0
...,...,...,...,...,...,...,...,...,...
408,0.20,100.0,210.0,50.0,2.48,NaN,0,1,0
409,0.10,100.0,210.0,50.0,1.51,NaN,0,1,0
410,0.30,100.0,210.0,50.0,6.76,NaN,0,1,0
411,0.30,100.0,210.0,50.0,9.08,NaN,0,1,0


In [21]:
knn_1 = KNNImputer(n_neighbors=3)
W = knn_1.fit_transform(zero_miss_WConc)
V = pd.DataFrame(knn_1.transform(D))

In [22]:
V.columns = zero_miss_WConc.columns
V

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_concentric,material_abs,material_pla,material_pla-aluminum
0,0.20,30.0,210.0,30.0,42.70,0.0,0.0,1.0,0.0
1,0.15,20.0,190.0,75.0,26.20,0.0,0.0,1.0,0.0
2,0.20,10.0,190.0,30.0,54.00,0.0,0.0,1.0,0.0
3,0.20,20.0,200.0,75.0,30.50,0.0,0.0,1.0,0.0
4,0.10,10.0,190.0,120.0,25.00,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
112,0.20,100.0,210.0,50.0,2.48,0.0,0.0,1.0,0.0
113,0.10,100.0,210.0,50.0,1.51,0.0,0.0,1.0,0.0
114,0.30,100.0,210.0,50.0,6.76,0.0,0.0,1.0,0.0
115,0.30,100.0,210.0,50.0,9.08,0.0,0.0,1.0,0.0


In [23]:
zero_miss_WGrid =  zero_missing_1H.drop(columns = [ 'infill_pattern_concentric','infill_pattern_honeycomb', 'infill_pattern_line',
       'infill_pattern_rectilinear', 'infill_pattern_solid','infill_pattern_tetrahedral', 'infill_pattern_triangle',
       'infill_pattern_zigzag'])
zero_miss_WGrid

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_grid,material_abs,material_pla,material_pla-aluminum
0,0.02,90.0,220.0,40.0,25.00,1.0,1.0,0.0,0.0
1,0.02,90.0,225.0,40.0,32.00,0.0,1.0,0.0,0.0
2,0.02,80.0,230.0,40.0,40.00,1.0,1.0,0.0,0.0
3,0.02,70.0,240.0,40.0,68.00,0.0,1.0,0.0,0.0
4,0.02,90.0,250.0,40.0,92.00,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
273,0.15,10.0,225.0,80.0,16.17,1.0,0.0,1.0,0.0
274,0.15,10.0,225.0,80.0,17.12,1.0,0.0,1.0,0.0
275,0.15,10.0,225.0,80.0,12.75,0.0,0.0,1.0,0.0
276,0.15,10.0,225.0,80.0,18.80,0.0,0.0,1.0,0.0


In [24]:
E = C.drop(columns='infill_pattern_concentric')
E['infill_pattern_grid'] = np.NaN
E = E.reindex(columns=list(zero_miss_WGrid.columns))
E

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_grid,material_abs,material_pla,material_pla-aluminum
190,0.20,30.0,210.0,30.0,42.7,NaN,0,1,0
191,0.15,20.0,190.0,75.0,26.2,NaN,0,1,0
192,0.20,10.0,190.0,30.0,54,NaN,0,1,0
193,0.20,20.0,200.0,75.0,30.5,NaN,0,1,0
194,0.10,10.0,190.0,120.0,25,NaN,0,1,0
...,...,...,...,...,...,...,...,...,...
408,0.20,100.0,210.0,50.0,2.48,NaN,0,1,0
409,0.10,100.0,210.0,50.0,1.51,NaN,0,1,0
410,0.30,100.0,210.0,50.0,6.76,NaN,0,1,0
411,0.30,100.0,210.0,50.0,9.08,NaN,0,1,0


In [25]:
knn_2 = KNNImputer(n_neighbors = 3)
U = knn_2.fit_transform(zero_miss_WGrid)
T = pd.DataFrame(knn_2.transform(E))
T.columns = zero_miss_WGrid.columns
T

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_grid,material_abs,material_pla,material_pla-aluminum
0,0.20,30.0,210.0,30.0,42.70,0.333333,0.0,1.0,0.0
1,0.15,20.0,190.0,75.0,26.20,1.000000,0.0,1.0,0.0
2,0.20,10.0,190.0,30.0,54.00,0.333333,0.0,1.0,0.0
3,0.20,20.0,200.0,75.0,30.50,1.000000,0.0,1.0,0.0
4,0.10,10.0,190.0,120.0,25.00,0.000000,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
112,0.20,100.0,210.0,50.0,2.48,0.000000,0.0,1.0,0.0
113,0.10,100.0,210.0,50.0,1.51,0.000000,0.0,1.0,0.0
114,0.30,100.0,210.0,50.0,6.76,0.000000,0.0,1.0,0.0
115,0.30,100.0,210.0,50.0,9.08,0.000000,0.0,1.0,0.0


In [26]:
zero_miss_Whoney = zero_missing_1H.drop(columns = [ 'infill_pattern_grid','infill_pattern_concentric', 'infill_pattern_line',
       'infill_pattern_rectilinear', 'infill_pattern_solid','infill_pattern_tetrahedral', 'infill_pattern_triangle',
       'infill_pattern_zigzag'])
zero_miss_Whoney

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_honeycomb,material_abs,material_pla,material_pla-aluminum
0,0.02,90.0,220.0,40.0,25.00,0.0,1.0,0.0,0.0
1,0.02,90.0,225.0,40.0,32.00,1.0,1.0,0.0,0.0
2,0.02,80.0,230.0,40.0,40.00,0.0,1.0,0.0,0.0
3,0.02,70.0,240.0,40.0,68.00,1.0,1.0,0.0,0.0
4,0.02,90.0,250.0,40.0,92.00,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
273,0.15,10.0,225.0,80.0,16.17,0.0,0.0,1.0,0.0
274,0.15,10.0,225.0,80.0,17.12,0.0,0.0,1.0,0.0
275,0.15,10.0,225.0,80.0,12.75,0.0,0.0,1.0,0.0
276,0.15,10.0,225.0,80.0,18.80,0.0,0.0,1.0,0.0


In [27]:
h = E.drop(columns='infill_pattern_grid')
h['infill_pattern_honeycomb'] = np.NaN
h = E.reindex(columns=list(zero_miss_Whoney.columns))
h


,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_honeycomb,material_abs,material_pla,material_pla-aluminum
190,0.20,30.0,210.0,30.0,42.7,NaN,0,1,0
191,0.15,20.0,190.0,75.0,26.2,NaN,0,1,0
192,0.20,10.0,190.0,30.0,54,NaN,0,1,0
193,0.20,20.0,200.0,75.0,30.5,NaN,0,1,0
194,0.10,10.0,190.0,120.0,25,NaN,0,1,0
...,...,...,...,...,...,...,...,...,...
408,0.20,100.0,210.0,50.0,2.48,NaN,0,1,0
409,0.10,100.0,210.0,50.0,1.51,NaN,0,1,0
410,0.30,100.0,210.0,50.0,6.76,NaN,0,1,0
411,0.30,100.0,210.0,50.0,9.08,NaN,0,1,0


In [28]:
knn_3 = KNNImputer(n_neighbors = 3)
h_1 = knn_3.fit_transform(zero_miss_Whoney)
h_2 = pd.DataFrame(knn_3.transform(h))
h_2.columns = zero_miss_Whoney.columns
h_2

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_honeycomb,material_abs,material_pla,material_pla-aluminum
0,0.20,30.0,210.0,30.0,42.70,0.666667,0.0,1.0,0.0
1,0.15,20.0,190.0,75.0,26.20,0.000000,0.0,1.0,0.0
2,0.20,10.0,190.0,30.0,54.00,0.666667,0.0,1.0,0.0
3,0.20,20.0,200.0,75.0,30.50,0.000000,0.0,1.0,0.0
4,0.10,10.0,190.0,120.0,25.00,0.000000,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
112,0.20,100.0,210.0,50.0,2.48,0.000000,0.0,1.0,0.0
113,0.10,100.0,210.0,50.0,1.51,0.000000,0.0,1.0,0.0
114,0.30,100.0,210.0,50.0,6.76,0.000000,0.0,1.0,0.0
115,0.30,100.0,210.0,50.0,9.08,0.000000,0.0,1.0,0.0


In [29]:
h_2.mean()

layer_height                  0.166667
infill_density               53.418803
nozzle_temperature          221.837607
print_speed                  60.726496
roughness                    17.274615
infill_pattern_honeycomb      0.094017
material_abs                  0.333333
material_pla                  0.666667
material_pla-aluminum         0.000000
dtype: float64

In [30]:
zero_miss_Wline = zero_missing_1H.drop(columns = [ 'infill_pattern_grid','infill_pattern_concentric', 'infill_pattern_honeycomb',
       'infill_pattern_rectilinear', 'infill_pattern_solid','infill_pattern_tetrahedral', 'infill_pattern_triangle',
       'infill_pattern_zigzag'])
zero_miss_Wline

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_line,material_abs,material_pla,material_pla-aluminum
0,0.02,90.0,220.0,40.0,25.00,0.0,1.0,0.0,0.0
1,0.02,90.0,225.0,40.0,32.00,0.0,1.0,0.0,0.0
2,0.02,80.0,230.0,40.0,40.00,0.0,1.0,0.0,0.0
3,0.02,70.0,240.0,40.0,68.00,0.0,1.0,0.0,0.0
4,0.02,90.0,250.0,40.0,92.00,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
273,0.15,10.0,225.0,80.0,16.17,0.0,0.0,1.0,0.0
274,0.15,10.0,225.0,80.0,17.12,0.0,0.0,1.0,0.0
275,0.15,10.0,225.0,80.0,12.75,0.0,0.0,1.0,0.0
276,0.15,10.0,225.0,80.0,18.80,0.0,0.0,1.0,0.0


In [31]:
L = h.drop(columns='infill_pattern_honeycomb')
L['infill_pattern_line'] = np.NaN
L = E.reindex(columns=list(zero_miss_Wline.columns))
L


,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_line,material_abs,material_pla,material_pla-aluminum
190,0.20,30.0,210.0,30.0,42.7,NaN,0,1,0
191,0.15,20.0,190.0,75.0,26.2,NaN,0,1,0
192,0.20,10.0,190.0,30.0,54,NaN,0,1,0
193,0.20,20.0,200.0,75.0,30.5,NaN,0,1,0
194,0.10,10.0,190.0,120.0,25,NaN,0,1,0
...,...,...,...,...,...,...,...,...,...
408,0.20,100.0,210.0,50.0,2.48,NaN,0,1,0
409,0.10,100.0,210.0,50.0,1.51,NaN,0,1,0
410,0.30,100.0,210.0,50.0,6.76,NaN,0,1,0
411,0.30,100.0,210.0,50.0,9.08,NaN,0,1,0


In [32]:
knn_4 = KNNImputer(n_neighbors = 3)
l_1 = knn_4.fit_transform(zero_miss_Wline)
l_2 = pd.DataFrame(knn_4.transform(L))
l_2.columns = zero_miss_Wline.columns
l_2

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_line,material_abs,material_pla,material_pla-aluminum
0,0.20,30.0,210.0,30.0,42.70,0.0,0.0,1.0,0.0
1,0.15,20.0,190.0,75.0,26.20,0.0,0.0,1.0,0.0
2,0.20,10.0,190.0,30.0,54.00,0.0,0.0,1.0,0.0
3,0.20,20.0,200.0,75.0,30.50,0.0,0.0,1.0,0.0
4,0.10,10.0,190.0,120.0,25.00,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
112,0.20,100.0,210.0,50.0,2.48,1.0,0.0,1.0,0.0
113,0.10,100.0,210.0,50.0,1.51,1.0,0.0,1.0,0.0
114,0.30,100.0,210.0,50.0,6.76,1.0,0.0,1.0,0.0
115,0.30,100.0,210.0,50.0,9.08,1.0,0.0,1.0,0.0


In [33]:
l_2.mean()

layer_height               0.166667
infill_density            53.418803
nozzle_temperature       221.837607
print_speed               60.726496
roughness                 17.274615
infill_pattern_line        0.205128
material_abs               0.333333
material_pla               0.666667
material_pla-aluminum      0.000000
dtype: float64

In [34]:
zero_miss_Winfill_pattern_rectilinear = zero_missing_1H.drop(columns = [ 'infill_pattern_grid','infill_pattern_concentric', 'infill_pattern_honeycomb',
       'infill_pattern_line', 'infill_pattern_solid','infill_pattern_tetrahedral', 'infill_pattern_triangle',
       'infill_pattern_zigzag'])
zero_miss_Winfill_pattern_rectilinear

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_rectilinear,material_abs,material_pla,material_pla-aluminum
0,0.02,90.0,220.0,40.0,25.00,0.0,1.0,0.0,0.0
1,0.02,90.0,225.0,40.0,32.00,0.0,1.0,0.0,0.0
2,0.02,80.0,230.0,40.0,40.00,0.0,1.0,0.0,0.0
3,0.02,70.0,240.0,40.0,68.00,0.0,1.0,0.0,0.0
4,0.02,90.0,250.0,40.0,92.00,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
273,0.15,10.0,225.0,80.0,16.17,0.0,0.0,1.0,0.0
274,0.15,10.0,225.0,80.0,17.12,0.0,0.0,1.0,0.0
275,0.15,10.0,225.0,80.0,12.75,0.0,0.0,1.0,0.0
276,0.15,10.0,225.0,80.0,18.80,0.0,0.0,1.0,0.0


In [35]:
R = L.drop(columns='infill_pattern_line')
R['infill_pattern_rectilinear'] = np.NaN
R = E.reindex(columns=list(zero_miss_Winfill_pattern_rectilinear.columns))
R

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_rectilinear,material_abs,material_pla,material_pla-aluminum
190,0.20,30.0,210.0,30.0,42.7,NaN,0,1,0
191,0.15,20.0,190.0,75.0,26.2,NaN,0,1,0
192,0.20,10.0,190.0,30.0,54,NaN,0,1,0
193,0.20,20.0,200.0,75.0,30.5,NaN,0,1,0
194,0.10,10.0,190.0,120.0,25,NaN,0,1,0
...,...,...,...,...,...,...,...,...,...
408,0.20,100.0,210.0,50.0,2.48,NaN,0,1,0
409,0.10,100.0,210.0,50.0,1.51,NaN,0,1,0
410,0.30,100.0,210.0,50.0,6.76,NaN,0,1,0
411,0.30,100.0,210.0,50.0,9.08,NaN,0,1,0


In [36]:
knn_5 = KNNImputer(n_neighbors = 3)
R_2 = knn_5.fit_transform(zero_miss_Winfill_pattern_rectilinear)
R_2 = pd.DataFrame(knn_5.transform(R))
R_2.columns = zero_miss_Winfill_pattern_rectilinear.columns
R_2

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_rectilinear,material_abs,material_pla,material_pla-aluminum
0,0.20,30.0,210.0,30.0,42.70,0.000000,0.0,1.0,0.0
1,0.15,20.0,190.0,75.0,26.20,0.000000,0.0,1.0,0.0
2,0.20,10.0,190.0,30.0,54.00,0.000000,0.0,1.0,0.0
3,0.20,20.0,200.0,75.0,30.50,0.000000,0.0,1.0,0.0
4,0.10,10.0,190.0,120.0,25.00,0.333333,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
112,0.20,100.0,210.0,50.0,2.48,0.000000,0.0,1.0,0.0
113,0.10,100.0,210.0,50.0,1.51,0.000000,0.0,1.0,0.0
114,0.30,100.0,210.0,50.0,6.76,0.000000,0.0,1.0,0.0
115,0.30,100.0,210.0,50.0,9.08,0.000000,0.0,1.0,0.0


In [37]:
R_2.mean()

layer_height                    0.166667
infill_density                 53.418803
nozzle_temperature            221.837607
print_speed                    60.726496
roughness                      17.274615
infill_pattern_rectilinear      0.011396
material_abs                    0.333333
material_pla                    0.666667
material_pla-aluminum           0.000000
dtype: float64

In [38]:
zero_miss_Winfill_pattern_solid = zero_missing_1H.drop(columns = [ 'infill_pattern_grid','infill_pattern_concentric', 'infill_pattern_honeycomb',
       'infill_pattern_line', 'infill_pattern_rectilinear','infill_pattern_tetrahedral', 'infill_pattern_triangle',
       'infill_pattern_zigzag'])
zero_miss_Winfill_pattern_solid

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_solid,material_abs,material_pla,material_pla-aluminum
0,0.02,90.0,220.0,40.0,25.00,0.0,1.0,0.0,0.0
1,0.02,90.0,225.0,40.0,32.00,0.0,1.0,0.0,0.0
2,0.02,80.0,230.0,40.0,40.00,0.0,1.0,0.0,0.0
3,0.02,70.0,240.0,40.0,68.00,0.0,1.0,0.0,0.0
4,0.02,90.0,250.0,40.0,92.00,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
273,0.15,10.0,225.0,80.0,16.17,0.0,0.0,1.0,0.0
274,0.15,10.0,225.0,80.0,17.12,0.0,0.0,1.0,0.0
275,0.15,10.0,225.0,80.0,12.75,0.0,0.0,1.0,0.0
276,0.15,10.0,225.0,80.0,18.80,0.0,0.0,1.0,0.0


In [39]:
S = R.drop(columns='infill_pattern_rectilinear')
S['infill_pattern_solid'] = np.NaN
S = E.reindex(columns=list(zero_miss_Winfill_pattern_solid.columns))
S

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_solid,material_abs,material_pla,material_pla-aluminum
190,0.20,30.0,210.0,30.0,42.7,NaN,0,1,0
191,0.15,20.0,190.0,75.0,26.2,NaN,0,1,0
192,0.20,10.0,190.0,30.0,54,NaN,0,1,0
193,0.20,20.0,200.0,75.0,30.5,NaN,0,1,0
194,0.10,10.0,190.0,120.0,25,NaN,0,1,0
...,...,...,...,...,...,...,...,...,...
408,0.20,100.0,210.0,50.0,2.48,NaN,0,1,0
409,0.10,100.0,210.0,50.0,1.51,NaN,0,1,0
410,0.30,100.0,210.0,50.0,6.76,NaN,0,1,0
411,0.30,100.0,210.0,50.0,9.08,NaN,0,1,0


In [40]:
knn_6 = KNNImputer(n_neighbors = 3)
S_2 = knn_6.fit_transform(zero_miss_Winfill_pattern_solid)
S_2 = pd.DataFrame(knn_6.transform(S))
S_2.columns = zero_miss_Winfill_pattern_solid.columns
S_2

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_solid,material_abs,material_pla,material_pla-aluminum
0,0.20,30.0,210.0,30.0,42.70,0.0,0.0,1.0,0.0
1,0.15,20.0,190.0,75.0,26.20,0.0,0.0,1.0,0.0
2,0.20,10.0,190.0,30.0,54.00,0.0,0.0,1.0,0.0
3,0.20,20.0,200.0,75.0,30.50,0.0,0.0,1.0,0.0
4,0.10,10.0,190.0,120.0,25.00,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
112,0.20,100.0,210.0,50.0,2.48,0.0,0.0,1.0,0.0
113,0.10,100.0,210.0,50.0,1.51,0.0,0.0,1.0,0.0
114,0.30,100.0,210.0,50.0,6.76,0.0,0.0,1.0,0.0
115,0.30,100.0,210.0,50.0,9.08,0.0,0.0,1.0,0.0


In [41]:

S_2.mean()

layer_height               0.166667
infill_density            53.418803
nozzle_temperature       221.837607
print_speed               60.726496
roughness                 17.274615
infill_pattern_solid       0.133903
material_abs               0.333333
material_pla               0.666667
material_pla-aluminum      0.000000
dtype: float64

In [42]:
zero_miss_Winfill_pattern_tetrahedral = zero_missing_1H.drop(columns = [ 'infill_pattern_grid','infill_pattern_concentric', 'infill_pattern_honeycomb',
       'infill_pattern_line', 'infill_pattern_rectilinear','infill_pattern_solid', 'infill_pattern_triangle',
       'infill_pattern_zigzag'])
zero_miss_Winfill_pattern_tetrahedral

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_tetrahedral,material_abs,material_pla,material_pla-aluminum
0,0.02,90.0,220.0,40.0,25.00,0.0,1.0,0.0,0.0
1,0.02,90.0,225.0,40.0,32.00,0.0,1.0,0.0,0.0
2,0.02,80.0,230.0,40.0,40.00,0.0,1.0,0.0,0.0
3,0.02,70.0,240.0,40.0,68.00,0.0,1.0,0.0,0.0
4,0.02,90.0,250.0,40.0,92.00,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
273,0.15,10.0,225.0,80.0,16.17,0.0,0.0,1.0,0.0
274,0.15,10.0,225.0,80.0,17.12,0.0,0.0,1.0,0.0
275,0.15,10.0,225.0,80.0,12.75,0.0,0.0,1.0,0.0
276,0.15,10.0,225.0,80.0,18.80,0.0,0.0,1.0,0.0


In [43]:
TH = S.drop(columns='infill_pattern_solid')
TH['infill_pattern_tetrahedral'] = np.NaN
TH = E.reindex(columns=list(zero_miss_Winfill_pattern_tetrahedral.columns))
TH

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_tetrahedral,material_abs,material_pla,material_pla-aluminum
190,0.20,30.0,210.0,30.0,42.7,NaN,0,1,0
191,0.15,20.0,190.0,75.0,26.2,NaN,0,1,0
192,0.20,10.0,190.0,30.0,54,NaN,0,1,0
193,0.20,20.0,200.0,75.0,30.5,NaN,0,1,0
194,0.10,10.0,190.0,120.0,25,NaN,0,1,0
...,...,...,...,...,...,...,...,...,...
408,0.20,100.0,210.0,50.0,2.48,NaN,0,1,0
409,0.10,100.0,210.0,50.0,1.51,NaN,0,1,0
410,0.30,100.0,210.0,50.0,6.76,NaN,0,1,0
411,0.30,100.0,210.0,50.0,9.08,NaN,0,1,0


In [44]:
knn_7 = KNNImputer(n_neighbors = 3)
TH_2 = knn_7.fit_transform(zero_miss_Winfill_pattern_tetrahedral)
TH_2 = pd.DataFrame(knn_7.transform(TH))
TH_2.columns = zero_miss_Winfill_pattern_tetrahedral.columns
TH_2

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_tetrahedral,material_abs,material_pla,material_pla-aluminum
0,0.20,30.0,210.0,30.0,42.70,0.0,0.0,1.0,0.0
1,0.15,20.0,190.0,75.0,26.20,0.0,0.0,1.0,0.0
2,0.20,10.0,190.0,30.0,54.00,0.0,0.0,1.0,0.0
3,0.20,20.0,200.0,75.0,30.50,0.0,0.0,1.0,0.0
4,0.10,10.0,190.0,120.0,25.00,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
112,0.20,100.0,210.0,50.0,2.48,0.0,0.0,1.0,0.0
113,0.10,100.0,210.0,50.0,1.51,0.0,0.0,1.0,0.0
114,0.30,100.0,210.0,50.0,6.76,0.0,0.0,1.0,0.0
115,0.30,100.0,210.0,50.0,9.08,0.0,0.0,1.0,0.0


In [45]:
TH_2.mean()

layer_height                    0.166667
infill_density                 53.418803
nozzle_temperature            221.837607
print_speed                    60.726496
roughness                      17.274615
infill_pattern_tetrahedral      0.051282
material_abs                    0.333333
material_pla                    0.666667
material_pla-aluminum           0.000000
dtype: float64

In [46]:
zero_miss_Winfill_pattern_triangle = zero_missing_1H.drop(columns = [ 'infill_pattern_grid','infill_pattern_concentric', 'infill_pattern_honeycomb',
       'infill_pattern_line', 'infill_pattern_rectilinear','infill_pattern_tetrahedral', 'infill_pattern_solid',
       'infill_pattern_zigzag'])
Tr = S.drop(columns='infill_pattern_solid')
Tr['infill_pattern_triangle'] = np.NaN
Tr = E.reindex(columns=list(zero_miss_Winfill_pattern_triangle.columns))
knn_8 = KNNImputer(n_neighbors = 3)
Tr_2 = knn_8.fit_transform(zero_miss_Winfill_pattern_triangle)
Tr_2 = pd.DataFrame(knn_8.transform(Tr))
Tr_2.columns = zero_miss_Winfill_pattern_triangle.columns
Tr_2

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_triangle,material_abs,material_pla,material_pla-aluminum
0,0.20,30.0,210.0,30.0,42.70,0.0,0.0,1.0,0.0
1,0.15,20.0,190.0,75.0,26.20,0.0,0.0,1.0,0.0
2,0.20,10.0,190.0,30.0,54.00,0.0,0.0,1.0,0.0
3,0.20,20.0,200.0,75.0,30.50,0.0,0.0,1.0,0.0
4,0.10,10.0,190.0,120.0,25.00,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
112,0.20,100.0,210.0,50.0,2.48,0.0,0.0,1.0,0.0
113,0.10,100.0,210.0,50.0,1.51,0.0,0.0,1.0,0.0
114,0.30,100.0,210.0,50.0,6.76,0.0,0.0,1.0,0.0
115,0.30,100.0,210.0,50.0,9.08,0.0,0.0,1.0,0.0


In [47]:
zero_miss_Winfill_pattern_zigzag = zero_missing_1H.drop(columns = [ 'infill_pattern_grid','infill_pattern_concentric', 'infill_pattern_honeycomb',
       'infill_pattern_line', 'infill_pattern_rectilinear','infill_pattern_tetrahedral', 'infill_pattern_solid',
       'infill_pattern_triangle'])
Z = TH.drop(columns='infill_pattern_tetrahedral')
Z['infill_pattern_zigzag'] = np.NaN
Z = E.reindex(columns=list(zero_miss_Winfill_pattern_zigzag.columns))
knn_9 = KNNImputer(n_neighbors = 3)
Z_2 = knn_9.fit_transform(zero_miss_Winfill_pattern_zigzag)
Z_2 = pd.DataFrame(knn_9.transform(Z))
Z_2.columns = zero_miss_Winfill_pattern_zigzag.columns
Z_2

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_zigzag,material_abs,material_pla,material_pla-aluminum
0,0.20,30.0,210.0,30.0,42.70,0.000000,0.0,1.0,0.0
1,0.15,20.0,190.0,75.0,26.20,0.000000,0.0,1.0,0.0
2,0.20,10.0,190.0,30.0,54.00,0.000000,0.0,1.0,0.0
3,0.20,20.0,200.0,75.0,30.50,0.000000,0.0,1.0,0.0
4,0.10,10.0,190.0,120.0,25.00,0.666667,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
112,0.20,100.0,210.0,50.0,2.48,0.000000,0.0,1.0,0.0
113,0.10,100.0,210.0,50.0,1.51,0.000000,0.0,1.0,0.0
114,0.30,100.0,210.0,50.0,6.76,0.000000,0.0,1.0,0.0
115,0.30,100.0,210.0,50.0,9.08,0.000000,0.0,1.0,0.0


In [48]:
zero_missing_3 = V.copy(0)
zero_missing_3['infill_pattern_grid'] = T['infill_pattern_grid']
zero_missing_3['infill_pattern_honeycomb'] = h_2['infill_pattern_honeycomb']
zero_missing_3['infill_pattern_line'] = l_2['infill_pattern_line']
zero_missing_3['infill_pattern_rectilinear'] = R_2['infill_pattern_rectilinear']
zero_missing_3['infill_pattern_solid'] = S_2['infill_pattern_solid']
zero_missing_3['infill_pattern_tetrahedral'] = TH_2['infill_pattern_tetrahedral']
zero_missing_3['infill_pattern_triangle'] = Tr_2['infill_pattern_triangle']
zero_missing_3['infill_pattern_zigzag'] = Z_2['infill_pattern_zigzag']
zero_missing_3_0 = zero_missing_3.reindex(columns=list(temp2.columns))
zero_missing_3_0

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_concentric,infill_pattern_grid,infill_pattern_honeycomb,infill_pattern_line,infill_pattern_rectilinear,infill_pattern_solid,infill_pattern_tetrahedral,infill_pattern_triangle,infill_pattern_zigzag,material_abs,material_pla,material_pla-aluminum
0,0.20,30.0,210.0,30.0,42.70,0.0,0.333333,0.666667,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,1.0,0.0
1,0.15,20.0,190.0,75.0,26.20,0.0,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,1.0,0.0
2,0.20,10.0,190.0,30.0,54.00,0.0,0.333333,0.666667,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,1.0,0.0
3,0.20,20.0,200.0,75.0,30.50,0.0,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,1.0,0.0
4,0.10,10.0,190.0,120.0,25.00,0.0,0.000000,0.000000,0.0,0.333333,0.0,0.0,0.0,0.666667,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,0.20,100.0,210.0,50.0,2.48,0.0,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.0,0.000000,0.0,1.0,0.0
113,0.10,100.0,210.0,50.0,1.51,0.0,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.0,0.000000,0.0,1.0,0.0
114,0.30,100.0,210.0,50.0,6.76,0.0,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.0,0.000000,0.0,1.0,0.0
115,0.30,100.0,210.0,50.0,9.08,0.0,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.0,0.000000,0.0,1.0,0.0


In [49]:
zero_missing_3_0.iloc[5,5:14][8] == zero_missing_3_0.iloc[5,5+8]

True

In [50]:
for r in range(117):
  a = zero_missing_3_0.iloc[r,5:14].max()
  for c in range(9):
    if zero_missing_3_0.iloc[r,5:14][c] == a:
      zero_missing_3_0.iat[r,5+c] = 1
    else:
      zero_missing_3_0.iat[r,5+c] = 0



In [52]:
zero_missing_3_0

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_concentric,infill_pattern_grid,infill_pattern_honeycomb,infill_pattern_line,infill_pattern_rectilinear,infill_pattern_solid,infill_pattern_tetrahedral,infill_pattern_triangle,infill_pattern_zigzag,material_abs,material_pla,material_pla-aluminum
0,0.20,30.0,210.0,30.0,42.70,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.15,20.0,190.0,75.0,26.20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.20,10.0,190.0,30.0,54.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.20,20.0,200.0,75.0,30.50,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.10,10.0,190.0,120.0,25.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,0.20,100.0,210.0,50.0,2.48,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
113,0.10,100.0,210.0,50.0,1.51,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
114,0.30,100.0,210.0,50.0,6.76,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
115,0.30,100.0,210.0,50.0,9.08,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [51]:
temp2

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_concentric,infill_pattern_grid,infill_pattern_honeycomb,infill_pattern_line,infill_pattern_rectilinear,infill_pattern_solid,infill_pattern_tetrahedral,infill_pattern_triangle,infill_pattern_zigzag,material_abs,material_pla,material_pla-aluminum
0,0.02,90.0,220.0,40.0,25.00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.02,90.0,225.0,40.0,32.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.02,80.0,230.0,40.0,40.00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.02,70.0,240.0,40.0,68.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.02,90.0,250.0,40.0,92.00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,0.15,10.0,225.0,80.0,16.17,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
274,0.15,10.0,225.0,80.0,17.12,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
275,0.15,10.0,225.0,80.0,12.75,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
276,0.15,10.0,225.0,80.0,18.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [53]:
combined_1 = pd.concat([zero_missing_3_0, temp2], axis=0)

In [80]:
combined_1.isnull().mean()
combined_1.columns
combined_1['material_nylon-aramid']=0
combined_1

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_concentric,infill_pattern_grid,infill_pattern_honeycomb,infill_pattern_line,infill_pattern_rectilinear,infill_pattern_solid,infill_pattern_tetrahedral,infill_pattern_triangle,infill_pattern_zigzag,material_abs,material_pla,material_pla-aluminum,material_nylon-aramid
0,0.20,30.0,210.0,30.0,42.70,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
1,0.15,20.0,190.0,75.0,26.20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
2,0.20,10.0,190.0,30.0,54.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
3,0.20,20.0,200.0,75.0,30.50,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
4,0.10,10.0,190.0,120.0,25.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,0.15,10.0,225.0,80.0,16.17,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
274,0.15,10.0,225.0,80.0,17.12,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
275,0.15,10.0,225.0,80.0,12.75,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
276,0.15,10.0,225.0,80.0,18.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0


In [81]:
temp_3 = pd.get_dummies(one_missing_print_speed, columns=['infill_pattern','material'], dtype=int)

In [82]:
temp_3


,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_grid,infill_pattern_honeycomb,infill_pattern_tetrahedral,material_nylon-aramid
50,0.2,70.0,280.0,NaN,1.88,0,0,1,1
51,0.2,80.0,280.0,NaN,1.81,0,0,1,1
52,0.2,90.0,280.0,NaN,1.63,0,0,1,1
53,0.2,70.0,290.0,NaN,2.13,0,1,0,1
54,0.2,80.0,290.0,NaN,2.06,0,1,0,1
55,0.2,90.0,290.0,NaN,1.92,0,1,0,1
56,0.2,70.0,300.0,NaN,2.48,1,0,0,1
57,0.2,80.0,300.0,NaN,2.34,1,0,0,1
58,0.2,90.0,300.0,NaN,2.15,1,0,0,1
59,0.3,70.0,280.0,NaN,3.03,1,0,0,1


In [83]:
temp_3[['infill_pattern_concentric', 'infill_pattern_line, infill_pattern_rectilinear', 'infill_pattern_solid', 'infill_pattern_triangle', 'infill_pattern_zigzag', 'material_abs', 'material_pla', 'material_pla-aluminum']] = 0
balla =['layer_height', 'infill_density', 'nozzle_temperature', 'print_speed',
       'roughness', 'infill_pattern_concentric', 'infill_pattern_grid',
       'infill_pattern_honeycomb', 'infill_pattern_line',
       'infill_pattern_rectilinear', 'infill_pattern_solid',
       'infill_pattern_tetrahedral', 'infill_pattern_triangle',
       'infill_pattern_zigzag', 'material_abs', 'material_pla',
       'material_pla-aluminum','material_nylon-aramid']

p =temp_3.reindex(columns= balla )
#p['material_nylon-aramid']=1
p

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_concentric,infill_pattern_grid,infill_pattern_honeycomb,infill_pattern_line,infill_pattern_rectilinear,infill_pattern_solid,infill_pattern_tetrahedral,infill_pattern_triangle,infill_pattern_zigzag,material_abs,material_pla,material_pla-aluminum,material_nylon-aramid
50,0.2,70.0,280.0,NaN,1.88,0,0,0,NaN,NaN,0,1,0,0,0,0,0,1
51,0.2,80.0,280.0,NaN,1.81,0,0,0,NaN,NaN,0,1,0,0,0,0,0,1
52,0.2,90.0,280.0,NaN,1.63,0,0,0,NaN,NaN,0,1,0,0,0,0,0,1
53,0.2,70.0,290.0,NaN,2.13,0,0,1,NaN,NaN,0,0,0,0,0,0,0,1
54,0.2,80.0,290.0,NaN,2.06,0,0,1,NaN,NaN,0,0,0,0,0,0,0,1
55,0.2,90.0,290.0,NaN,1.92,0,0,1,NaN,NaN,0,0,0,0,0,0,0,1
56,0.2,70.0,300.0,NaN,2.48,0,1,0,NaN,NaN,0,0,0,0,0,0,0,1
57,0.2,80.0,300.0,NaN,2.34,0,1,0,NaN,NaN,0,0,0,0,0,0,0,1
58,0.2,90.0,300.0,NaN,2.15,0,1,0,NaN,NaN,0,0,0,0,0,0,0,1
59,0.3,70.0,280.0,NaN,3.03,0,1,0,NaN,NaN,0,0,0,0,0,0,0,1


In [84]:
combined_2 = pd.concat([combined_1, p], axis=0)

In [85]:
combined_2

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_concentric,infill_pattern_grid,infill_pattern_honeycomb,infill_pattern_line,infill_pattern_rectilinear,infill_pattern_solid,infill_pattern_tetrahedral,infill_pattern_triangle,infill_pattern_zigzag,material_abs,material_pla,material_pla-aluminum,material_nylon-aramid
0,0.20,30.0,210.0,30.0,42.7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
1,0.15,20.0,190.0,75.0,26.2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
2,0.20,10.0,190.0,30.0,54.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
3,0.20,20.0,200.0,75.0,30.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
4,0.10,10.0,190.0,120.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,0.40,80.0,290.0,NaN,2.78,0.0,1.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
73,0.40,90.0,290.0,NaN,2.67,0.0,1.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
74,0.40,70.0,300.0,NaN,3.08,0.0,0.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1
75,0.40,80.0,300.0,NaN,2.4,0.0,0.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1


In [86]:
knn = KNNImputer(n_neighbors=3)
temp4 = pd.DataFrame(knn.fit_transform(combined_2))
temp4.columns = combined_2.columns
temp4

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_concentric,infill_pattern_grid,infill_pattern_honeycomb,infill_pattern_line,infill_pattern_rectilinear,infill_pattern_solid,infill_pattern_tetrahedral,infill_pattern_triangle,infill_pattern_zigzag,material_abs,material_pla,material_pla-aluminum,material_nylon-aramid
0,0.20,30.0,210.0,30.0,42.70,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.15,20.0,190.0,75.0,26.20,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.20,10.0,190.0,30.0,54.00,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.20,20.0,200.0,75.0,30.50,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.10,10.0,190.0,120.0,25.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,0.40,80.0,290.0,50.0,2.78,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
418,0.40,90.0,290.0,50.0,2.67,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
419,0.40,70.0,300.0,50.0,3.08,0.0,0.0,0.0,0.333333,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
420,0.40,80.0,300.0,50.0,2.40,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [88]:
temp4.isnull().mean()

layer_height                  0.0
infill_density                0.0
nozzle_temperature            0.0
print_speed                   0.0
roughness                     0.0
infill_pattern_concentric     0.0
infill_pattern_grid           0.0
infill_pattern_honeycomb      0.0
infill_pattern_line           0.0
infill_pattern_rectilinear    0.0
infill_pattern_solid          0.0
infill_pattern_tetrahedral    0.0
infill_pattern_triangle       0.0
infill_pattern_zigzag         0.0
material_abs                  0.0
material_pla                  0.0
material_pla-aluminum         0.0
material_nylon-aramid         0.0
dtype: float64